In [1]:
## Run when initialise the code
import os

import geopandas as gpd
import osmnx as ox
from geopandas import GeoDataFrame, GeoSeries
from osmnx import io

from pandas import DataFrame

project_crs = 'epsg:3857'
from sklearn.cluster import DBSCAN
from shapely.geometry import Point, LineString, MultiPolygon, MultiPoint
import math
import warnings
import pandas as pd

from tqdm import tqdm

warnings.filterwarnings(action='ignore')
from momepy import remove_false_nodes, extend_lines

pjr_loc = os.path.dirname(os.getcwd())
import ast  # to convert str with list to list of string
from itertools import combinations
import numpy as np
from math import log2

In [10]:

place = 'Tel_Aviv'

data_folder = f'{os.path.dirname(pjr_loc)}/places/more/{place}_test'
data_to_read = f'{os.path.dirname(pjr_loc)}/places/{place}'
os.makedirs(f'{data_folder}', exist_ok=True)

# Filter out polylines and calculate angles
my_gdf = gpd.read_file(f'{data_to_read}/osm_data.gpkg',
                       layer='edges')  # Identify roundabout elements, if any exist, and store them in a
my_gdf.rename(columns={'name:en': 'name'}, inplace=True)
is_junction = True if 'junction' in my_gdf.columns else False

if is_junction:
    round_about = my_gdf[my_gdf['junction'].isin(['roundabout', 'circular'])]
    my_gdf2 = my_gdf[~((my_gdf['junction'] == 'roundabout') | (my_gdf['junction'] == 'circular'))]
to_remove = my_gdf[~((my_gdf['highway'] == 'motorway') | (my_gdf['highway'] == 'trunk') | (
            my_gdf['highway'] == 'motorway_link') | (my_gdf['highway'] == 'motorway_link') | (
                                     my_gdf['highway'] == 'trunk_link'))]

df_pro = to_remove.to_crs(project_crs).dropna(subset=['name'])
df_pro = df_pro[df_pro['name'] != '']


'C:\\Users\\18059\\OneDrive - ariel.ac.il\\Current_research\\SOD/places/Tel_Aviv'

In [19]:
df_pro['angle'] = df_pro['bearing'].apply(lambda x: x if x < 180 else x - 180)
df_pro [df_pro ['name']=='Abba Ahimeir']

,u,v,key,osmid,name,highway,oneway,reversed,length,bearing,from,to,tunnel,junction,geometry,angle
10001,414719373,415474635,0,35373067,Abba Ahimeir,residential,False,True,94.108,280.2,415474635,414719373,,,"LINESTRING (3874356.133 3779661.439, 3874249.9...",100.2
10003,415474635,10719151044,0,35373067,Abba Ahimeir,residential,False,True,13.102,280.5,10719151044,415474635,,,"LINESTRING (3874371.361 3779658.613, 3874356.1...",100.5
10006,4970978703,3540731265,0,35417795,Abba Ahimeir,residential,False,False,170.855,280.9,3540731265,4970978703,,,"LINESTRING (3874444.643 3779655.117, 3874390.0...",100.9
10007,414719375,1001172667,0,35372970,Abba Ahimeir,tertiary,False,False,31.159,93.7,1001172667,414719375,,,"LINESTRING (3874177.409 3779687.347, 3874210.4...",93.7
10010,1001172667,10719124394,0,35372970,Abba Ahimeir,tertiary,False,False,74.236,93.7,10719124394,1001172667,,,"LINESTRING (3874089.834 3779692.973, 3874177.4...",93.7
10013,414719517,5894568356,0,35372970,Abba Ahimeir,tertiary,False,True,2.926,273.9,5894568356,414719517,,,"LINESTRING (3874034.241 3779696.535, 3874030.7...",93.9
10016,5894568356,10719124394,0,35372970,Abba Ahimeir,tertiary,False,True,47.125,273.7,10719124394,5894568356,,,"LINESTRING (3874089.834 3779692.973, 3874034.2...",93.7
10018,4970978701,1163053962,0,35372967,Abba Ahimeir,tertiary,False,False,157.755,275.1,1163053962,4970978701,,,"LINESTRING (3874217.729 3779697.192, 3874031.9...",95.1
10024,414719543,415448758,0,35373057,Abba Ahimeir,tertiary,False,True,245.491,93.2,415448758,414719543,,,"LINESTRING (3873715.990 3779734.352, 3874005.7...",93.2
10028,414719544,1001172607,0,35373059,Abba Ahimeir,tertiary,False,False,57.126,94.3,1001172607,414719544,,,"LINESTRING (3873934.365 3779705.959, 3874001.7...",94.3


In [8]:
#